# BLU03 - Exercises Notebook

In [1]:
import hashlib # for grading purposes
import math
import numpy as np
import pandas as pd
import requests
import sqlalchemy

from bs4 import BeautifulSoup

## Part A - SQL exercises

### Querying the FIFAdb with a SQL client

Open your favorite SQL client and connect to the FIFAdb.
The connection settings are the following.

* host: batch4-s02-db-instance.ctq2kxc7kx1i.eu-west-1.rds.amazonaws.com
* port: 5432
* user: ldsa_student
* database: batch4_s02_db
* schema: public
* password: XXX (shared through slack)

This is a different schema than the one we used in the learning notebooks (don't forget to change to this schema, see the Learning Notebook). This schema contains information about football matches, players, teams, and which league and country these matches took place in. Additionally, it also contains the player's and team's "attributes", sourced from the EA Sports' FIFA video game series.

The tables in this schema are the following:

1. Match: has information about the football matches: who were the 11 home and away players (identified by their player_id), how many goals did each team score, the date of the match, the league id and the home/away team id's.
2. Player: contains informations about the players.
3. Team: contains information about the teams.
4. League: contains information about the football leagues, including the id of the country where they take place.
5. Country: names and id's of the countries
6. Player_Attributes: contains the attributes for each player.
7. Team_Attributes: contains the attributes for each team.

You can preview these tables using the SQL client.

### Q1. Select the name of the team with id 10252

Write a query that selects the name of the team whose id is 10252, and run it in the SQL client.

Then, assign the result to variable q1_answer (just copy and paste the name you obtained).

In [2]:
# YOUR CODE HERE
q1_answer = 'Aston Villa'
#raise NotImplementedError()

In [3]:
expected_hash = '2fa9bd880a4d7139b4a42c4adb80a942a8b4c7e4ee95aa0f13be5b4005995dcf'
assert hashlib.sha256(q1_answer.encode()).hexdigest() == expected_hash

### Q2. How many players are taller than 200 cm?

Write a query that counts how many players are taller than 200 cm.

Then, assign the result to variable q2_answer (just copy and paste the value).

In [4]:
# YOUR CODE HERE
q2_answer = 22
#raise NotImplementedError()

In [5]:
expected_hash = '785f3ec7eb32f30b90cd0fcf3657d388b5ff4297f2f9716ff66e9b69c05ddd09'
assert hashlib.sha256(str(q2_answer).encode()).hexdigest() == expected_hash

### Q3. Calculate the average buildUpPlaySpeed attribute of teams whose names start with "FC"

Do teams whose names start with "FC" usually play fast? Only one way to find out! 

Write a query that calculates the average buildUpPlaySpeed attribute of teams whose name is **like** "FC *something*" (for example, "FC Pandas"), and run it in the SQL client.

Then, assign the result to variable q3_answer, rounded to two decimal places!

**Hints**: check the [LIKE](https://www.postgresql.org/docs/current/static/functions-matching.html#FUNCTIONS-LIKE) keyword for this exercise. Also: the team name is not on the Team_Attributes table - you'll have to get it from somewhere else.

In [6]:
# YOUR CODE HERE
q3_answer = 51.32
#raise NotImplementedError()

In [7]:
expected_hash = '4ce1d6e46996014e34d90e3d0dcec6b8d0b4948173f4aaf37084033c3fa0a372'
assert hashlib.sha256(str(q3_answer).encode()).hexdigest() == expected_hash

### Q4. Count how many different teams have played in the "Italy Serie A" league

Write a query that counts the number of **distinct** teams that played in the league whose name is "Italy Serie A", across all games. You can calculate this value considering only the home or away team - it should be the same, because every team has played on both sides of the field!

Assign the result to variable q4_answer (just copy and paste the value).

**Hints**: keep in mind you only want to count DISTINCT team names. For this, the [DISTINCT](https://www.postgresql.org/docs/current/static/sql-select.html#SQL-DISTINCT) keyword will be essential. Also, remember that the relationship between Country and Match isn't explicitly presented on the Match table, but there is a relationship between League and Country.

In [8]:
# YOUR CODE HERE
q4_answer= 32
#raise NotImplementedError()

In [9]:
expected_hash = 'e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027aa13515e4f3884bb6b'
assert hashlib.sha256(str(q4_answer).encode()).hexdigest() == expected_hash

### Q5. Find out in what country the teams score an highest average number of goals when playing away.

Write a query to find out in what country the teams score the highest average number of goals when playing away.

Assign this country to variable q5_answer_1.

Also find out what this average amount of goals is, and assign it to the variable q5_answer_2 (round to one decimal place).

**Hint**: there isn't a direct connection between the matches and the country, but you can get there using an extra table.

In [10]:
# YOUR CODE HERE
q5_answer_1 = 'Netherlands'
q5_answer_2 =1.3
#raise NotImplementedError()

In [11]:
expected_hash_1 = '3a386c13d283dc2ebbe2aeffc1ab3930d086b28956930d913d8dc58f34c013b7'
assert hashlib.sha256(q5_answer_1.encode()).hexdigest() == expected_hash_1

expected_goals_hash = '9c193c604ad7de942961af97b39ff541f2e611fdf0b93a3044e16dfbd808f41b'
assert hashlib.sha256(str(q5_answer_2).encode()).hexdigest() == expected_goals_hash

### Querying the FIFAdb with pandas

In these exercises, the goal is to query the FIFAdb using pandas.

### Q6. Find the teams who are very successful at dribbling and at scoring goals.

The connection settings to use in this exercise are the same ones as in the previous exercises.

Write a query to find the name, short_name and *max amount of goals scored when playing at home* of the teams with a high "buildUpPlayDribbling" team attribute (*greater than 50*).

Search only for teams with:
* an *average amount of goals scored when playing at home* greater than 2; 
* more than 50 games played at home, to reduce the number of statistically insignificant results.

Order the results by the team short names in descending order.

Assign the result to dataframe df6.

In [12]:
# Create an engine that allows to connect to the FIFAdb PostgreSQL database
# engine = sqlalchemy.create_engine(...)
# Db settings - PostgreSQL
username = 'ldsa_student'
password = 'R4Fr4P3aAgMYBqqP'  # the password is not XXX by the way
host_name = 'batch4-s02-db-instance.ctq2kxc7kx1i.eu-west-1.rds.amazonaws.com'
port = 5432
db_name = 'batch4_s02_db'
schema = 'public'
conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)
conn_args = {'options': '-csearch_path={}'.format(schema)}
engine = sqlalchemy.create_engine(conn_str, connect_args=conn_args)
# YOUR CODE HERE
#raise NotImplementedError()
query = 'select t.name,t.short_name,max(m.home_team_goal) as max_amount_of_goal from team as t inner join team_attributes as ta on t.id = ta.team_id inner join match as m on m.home_team_id = t.id where ta.buildupplaydribbling>50 group by t.name,t.short_name having count(t.name)>50 and avg(m.home_team_goal)>2 order by t.short_name desc;'

# Write the query as specified in the question
# YOUR CODE HERE
#raise NotImplementedError()

# Use pandas read_sql_query function to read the query result into a DataFrame
df6 = pd.read_sql_query(query,engine)
df6
# YOUR CODE HERE
#raise NotImplementedError()

name short_name  max_amount_of_goal
0        BSC Young Boys         YB                   7
1         VfL Wolfsburg        WOL                   6
2        Real Madrid CF        REA                  10
3               Rangers        RAN                   7
4   Paris Saint-Germain        PSG                   6
5                Napoli        NAP                   6
6        Club Brugge KV        CLB                   7
7                Celtic        CEL                   9
8            SL Benfica        BEN                   8
9              FC Basel        BAS                   6
10              Arsenal        ARS                   7
11      Atlético Madrid        AMA                   7

In [13]:
assert type(engine) == sqlalchemy.engine.base.Engine
assert len(df6) == 12
assert len(df6.columns) == 3

expected_hash = '8df238f8673b71d0b4e74ce6a2439d63b31cefd33c2aeb795328f6a7b7a77cea'
assert hashlib.sha256(df6.iloc[2]["name"].encode()).hexdigest() == expected_hash

expected_hash = '4b1824b663a4f4b36465d3d43c70f19500a0f47feafd71722e1459673394e041'
assert hashlib.sha256(df6.iloc[4].short_name.encode()).hexdigest() == expected_hash

### Q7. Find out some attributes from players with high potential.

In this exercise, we want to query a local SQLite database.
In order to do this, connect to the FIFAdb.sqlite database, as was done in the learning notebooks for the_movies.db. The database file we're using is in the **data** directory.

Write a query that selects the player name, height, weight, potential, crossing and finishing for all players with overall_rating lower or equal to 55 and potential greater than 70. Order these results by player name in ascending order.

Use pandas to read this query into a DataFrame called df7 with six columns: name, height, weight, potential, crossing, and finishing.

In [14]:
# Create an engine that allows to to connect to the the_movies.db SQLite database
# Local SQLite Db
db_file_path = 'data/FIFAdb.sqlite'
conn_str = 'sqlite:///{}'.format(db_file_path)
engine = sqlalchemy.create_engine(conn_str)
# YOUR CODE HERE
#raise NotImplementedError()


# Write the query as specified in the question
query = 'select p.name, p.height, p.weight,pa.potential, pa.crossing, pa.finishing from player as p inner join player_attributes as pa on p.id = pa.player_id where pa.overall_rating <=55 and pa.potential >70 order by p.name asc'
# YOUR CODE HERE
#raise NotImplementedError()

# Use pandas read_sql_query function to read the query result into a DataFrame
df7 = pd.read_sql_query(query,engine)
# YOUR CODE HERE
#raise NotImplementedError()

In [15]:
assert type(engine) == sqlalchemy.engine.base.Engine
assert len(df7) == 16
assert len(df7.columns) == 6
assert df7.columns.tolist() == ['name', 'height', 'weight', 'potential', 'crossing', 'finishing']

expected_hash = 'e966c1725d9d0682476dd1959984b5d597af96ed9de4dd12e860c084ba68f2aa'
assert hashlib.sha256(df7.loc[0, 'name'].encode()).hexdigest() == expected_hash

expected_hash = '71c77c676e8d2d8f17b4ddcf203211ad6b887d96250270c397094bd19cf0b4f5'
assert hashlib.sha256(str(df7.loc[2, 'height']).encode()).hexdigest() == expected_hash

expected_hash = 'd3d6cae6e3de77d000864df356581fc373ed344126f5b707dbd3c7c48f39dbfd'
assert hashlib.sha256(str(df7.loc[7, 'crossing']).encode()).hexdigest() == expected_hash

## Part B - Public APIs

<h2>WARNING</h2>

The following API-related exercises have been disabled due to the API being down.

-----------------------------------

In this exercises, the goal is to get data from a public API. We'll go full geek, and use the [Magic: The Gathering API](https://magicthegathering.io/)!

In order to complete the exercises, you'll have to consult the API's [documentation](https://docs.magicthegathering.io/). More specifically, you'll have to check the API (v1) section to see what are the different endpoints from which you can GET information.

<br>

<img src="media/api-image.png" width=600>

<br>

### Q8. Find information about the Black Lotus card.

The "Black Lotus" card is one of the rarest cards in the game. Use the API to find information about this card. 

In order to get this data, you'll need to do an HTTP GET request to one of the endpoints. You also need to specify

* the card name: Black Lotus
* the set name: Collectors’ Edition (copy the set name from here, the tick in the name is not a regular tick)

using the right parameters.

The response should be converted to a JSON object (which is the same as a dictionary in Python), and assigned to variable q8_answer.

Note: the request might take a minute.

In [ ]:
# Do an HTTP GET request to the Magic: The Gathering API to get information about 
# the Black Lotus card as a JSON object

# payload = {
#     "name": "Black Lotus",
#     "setName": "Collectors’ Edition"
# }
# response = requests.get("https://api.magicthegathering.io/v1/cards", params=payload)
# q8_answer = response.json()


In [ ]:
# assert type(q8_answer) == dict
# assert 'cards' in q8_answer
# assert set(q8_answer['cards'][0].keys()) == {'artist', 'cmc', 'colorIdentity', 'colors', 'foreignNames', 'id', 'layout', 'legalities', 'manaCost', 'name', 'number', 'printings', 'rarity', 'rulings', 'set', 'setName', 'subtypes', 'supertypes', 'text', 'type', 'types'}

# card_name_hash = '790a4590717918485f0e38385da506c95ec4eb9b6adf42a0183c260c2c887ae2'
# assert hashlib.sha256(q8_answer['cards'][0]['name'].encode()).hexdigest() == card_name_hash

# expected_id_hash = '572e1ec08de73129296c63d1d475dbc8bf3c5178fda663e687cda5897dcf0cd4'
# assert hashlib.sha256(str(q8_answer['cards'][0]['id']).encode()).hexdigest() == expected_id_hash

### Q9. Find information about a specific set

Read the documentation of the API in order to find out how to request and filter sets, and get all information from the sets belonging to the "Shadows over Innistrad" block.

The desired results can be found in the 'sets' field of the response. Assign these results to the q9_answer variable.

In [ ]:
# # Do an HTTP GET request to filter the card sets according to the criteria above

# payload = {
#     "block": "Shadows over Innistrad"
# }
# response = requests.get("https://api.magicthegathering.io/v1/sets", params=payload)
# q9_answer = response.json()['sets']

In [ ]:
# assert type(q9_answer) == list
# assert len(q9_answer) == 4
# assert set(q9_answer[0].keys()) == {'block', 'booster', 'code', 'name', 'onlineOnly', 'releaseDate', 'type'}

# expected_name = '8bdf7b056442242035bcb6d8d88a3be4f8e1b2f306ccff681b64bba3f5ea4ee8'
# assert hashlib.sha256(q9_answer[0]['name'].encode()).hexdigest() == expected_name

# expected_date = 'f2b552e894e18156759c1f4d7eedbc0ca5a5d9c464c3f53a204d8947465d39fc'
# assert hashlib.sha256(q9_answer[1]['releaseDate'].encode()).hexdigest() == expected_date

## Part C - Web scraping

In this exercise, we're going to use web scraping to get data about some books! You can find them in this page: http://books.toscrape.com/

### Q10. Scrape all book categories from the sidebar.

Store the book categories found in the sidebar, on the left side of the page, in the variable **category_list**. This should be a list of strings with all the categories. There are several ways to reach this solution.
**Warning:** don't forget to remove the "Books" string found on the top of the sidebar from the list, since it's not a category. 

**Tips:** A good approach would be to "Inspect" the page and look for the class and tag of the sidebar.

In [16]:
# Assign the URL of the page to be scraped to variable url
url ='http://books.toscrape.com/'
# YOUR CODE HERE
#raise NotImplementedError()

# Do a GET request to get the page content, using the url we've just defined
response = requests.get(url)
# YOUR CODE HERE
#raise NotImplementedError()

# Instantiate a soup object using the response of the GET request
soup = BeautifulSoup(response.content, 'html.parser')
# YOUR CODE HERE
#raise NotImplementedError()
    
# Now it's the tricky part!
# Parse the soup in order to retrieve the categories.
# In the end, store the categories in a list and assign it to variable category_list.
soup_children = list(soup.children)
book_list = soup.find_all('ul', class_="nav nav-list")
book_table = soup.find_all('li',class_ = False)
book_names = [book.get_text().strip() for book in book_table]
book_names = book_names[2:]
category_list=book_names
# YOUR CODE HERE
#raise NotImplementedError()

In [17]:
assert 'Books' not in category_list, "Did you forget to remove the Books header?"

expected_hash = 'ded4efd711466185ecddc1fd70e5f1e5763731685515f717a05db6c75f3b3d0b'
assert hashlib.sha256(str(sorted(category_list)).encode()).hexdigest() == expected_hash

### Q11. Find all the prices in the first page

Use your web scraping skills to find all the prices in the first page! Assign them as **strings** (with the currency attached) to the variable price_list, 

**Food for thought:** notice what happens to the URL when you press the "next" button at the bottom of the page. Can you figure out a way to scrape the entire website? Don't forget to scrape responsibly, and limit your request rates!

In [19]:
# Assign the URL of the page to be scraped to variable url
url = 'http://books.toscrape.com/'
# YOUR CODE HERE
#raise NotImplementedError()

# Do a GET request to get the page content, using the url we've just defined
response = requests.get(url)
# YOUR CODE HERE
#raise NotImplementedError()

# Instanciate a soup object using the response of the GET request
# YOUR CODE HERE
soup = BeautifulSoup(response.content, 'html.parser')
#raise NotImplementedError()

# Parse the soup in order to retrieve the prices.
# In the end, store the prices as strings in a list and assign it to variable price_list.
price_list_all = soup.find_all('p', class_= "price_color") 
price_list = [p.get_text() for p in price_list_all]


# YOUR CODE HERE
#raise NotImplementedError()

In [20]:
expected_hash = '45449fc594578027c5c7ffc5ce11e85c93620f333de511e5a53e2a691359244f'
assert hashlib.sha256(str(sorted(price_list)).encode()).hexdigest() == expected_hash